# EXTRACTION
### webscraps inec database

In [318]:
#driver for Chrome
from selenium.webdriver import Chrome
#selector for dropdown
from selenium.webdriver.support.ui import Select
#waits until a process or page is accessible 
from selenium.webdriver.support.ui import WebDriverWait
#By.XPATH, By.CSS_SELECTOR etc
from selenium.webdriver.common.by import By
#manipulates properties of chrome driver like add fake user agent
from selenium.webdriver.chrome.options import Options
#certain conditions need to be met before a process begins
from selenium.webdriver.support import expected_conditions as EC
#this changes the useragent name which reduces reCAPTCHA verification
from fake_useragent import UserAgent
#allows code to sleep for a little time until process is completely done
import time

options = Options()
ua = UserAgent()
userAgent = ua.random
options.add_argument(f'user-agent={userAgent}')

#access to the path of your chromedriver.exe
driver = Chrome('C:/Users/HP/Downloads/inec_scum/chromedriver.exe')
#the homepage since you have to sign in first before navigating to the required page for sign in
driver.get("https://cvr.inecnigeria.org/Home/start")
#waits until your page loads completely before sign in process begins
driver.implicitly_wait(20)

#sign in cuse using xpath and the id of the html properties as key as well ass value for login button
driver.find_element_by_xpath("""//*[@id="LoginEmail"]""").send_keys('georgemichaeldagogo@gmail.com')
driver.find_element_by_xpath("""//*[@id="LoginPassword"]""").send_keys('ayabagreen1')
driver.find_element_by_xpath("""//*[@value="Log in"]""").click()

#navigates to the page for inputting state, LGA,AREA id and polling unit
driver.get("https://cvr.inecnigeria.org/VotersRegister")
driver.implicitly_wait(6)

select1 = Select(driver.find_element_by_id('VoterRegisterStateId'))
#selecting by value instead of state name
select1.select_by_value('1')
select2 = Select(driver.find_element_by_id('VoterRegisterLocalGovernmentId'))
select2.select_by_value('1')
select3 = Select(driver.find_element_by_id('VoterRegisterRegistrationAreaId'))
#at a point the values sudden increase exponentially like from 22 to 61373
select3.select_by_value('1')
select4 = Select(driver.find_element_by_id('VoterRegisterPollingUnitId'))
select4.select_by_value('1')

#waits for page is ready then switches to the reCAPTCHA iframe
WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,"//iframe[@title='reCAPTCHA']")))
#clicks on the reCAPTCHA button and it either verifies without image test or it doesn't. currently at 8 out of 10 times with 8 being verification without image test
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.recaptcha-checkbox-border"))).click()


time.sleep(5)
#switches from reCAPTCHA iframe back to default reCAPTCHA else your code will gives error about not recognizing the next xpath
driver.switch_to.default_content()
#clicks on display button
driver.find_element_by_xpath("""//*[@value="DISPLAY REGISTER"]""").click()
driver.implicitly_wait(20)
names = []
loca = []
pictures = []
pages = []
#required data can be found in the table with class name below that contains the name, gender, date of birth and vin for everyone in that page
contents =driver.find_elements_by_class_name("table.table-condensed.table-borderless.condensed-rows.voter")
for i in contents:
    name = i.get_attribute('innerText')
    names.append(name)
#required data can be found in the table with class name below that contains the lga, polling unit, area and delim for everyone
location =driver.find_elements_by_class_name("table.table-bordered.table-striped.afterpagebreak.bg-white")
for s in location:
    local= s.get_attribute('innerText')
    loca.append(local)
#to get the pictures of the registered voters under the img tag
picture = driver.find_elements_by_css_selector("img")
for p in picture:
    pic= p.get_attribute('src')
    pictures.append(pic)
#this is to get the extent of the pages like "page 1 of 50"
page = driver.find_elements_by_class_name("col-lg-12.text-right.nav-text")
for a in page:
    pag= a.text
    pages.append(pag)
    
def all_pages(web_page):
    """this function takes the next webpage https://cvr.inecnigeria.org/voters_register/index/display/page:2 as a
        parameter and returns the basic information of the registered voters in a polling unit as well as their pictures 
         and appends it to the universal list with the appopriate name
    """
    driver.get(web_page)
    contents =driver.find_elements_by_class_name("table.table-condensed.table-borderless.condensed-rows.voter")
    for i in contents:
        name = i.get_attribute('innerText')
        names.append(name)
    picture = driver.find_elements_by_css_selector("img")
    for p in picture:
        pic= p.get_attribute('src')
        pictures.append(pic)

#after raw extract of the pages the content was split
q =[item.split() for item in pages]
#the extent of the page number has the below index
page_no = q[0][5] 
#it must be an integar or else it can't be parsed into the variable list a that has a range 
page_num = int(page_no)
#this list is a formatted string that increases the page number at the end of the web address by 1 starting with 2 for the second page and the limit
# is the page extent number + 1 because python index doesn't account for the last number given
a = [f'https://cvr.inecnigeria.org/voters_register/index/display/page:{i:d}' for i in range(2, page_num +1 , 1)] 

#this loop uses the a variable list and runs a loop for every page increment using 
# the all_pages function that returns the basic info and pictures and appends it to the required list awaiting 
#  awaiting cleaning and wrangling
for o in a:
    all_pages(o)
#ends the driver process

driver.quit()

In [340]:
a

['https://cvr.inecnigeria.org/voters_register/index/display/page:2',
 'https://cvr.inecnigeria.org/voters_register/index/display/page:3',
 'https://cvr.inecnigeria.org/voters_register/index/display/page:4',
 'https://cvr.inecnigeria.org/voters_register/index/display/page:5',
 'https://cvr.inecnigeria.org/voters_register/index/display/page:6',
 'https://cvr.inecnigeria.org/voters_register/index/display/page:7',
 'https://cvr.inecnigeria.org/voters_register/index/display/page:8',
 'https://cvr.inecnigeria.org/voters_register/index/display/page:9',
 'https://cvr.inecnigeria.org/voters_register/index/display/page:10',
 'https://cvr.inecnigeria.org/voters_register/index/display/page:11',
 'https://cvr.inecnigeria.org/voters_register/index/display/page:12',
 'https://cvr.inecnigeria.org/voters_register/index/display/page:13',
 'https://cvr.inecnigeria.org/voters_register/index/display/page:14',
 'https://cvr.inecnigeria.org/voters_register/index/display/page:15',
 'https://cvr.inecnigeria.or

# TRANSFORM

In [319]:
import pandas as pd
#turns the area list into a pandas dataframe and changes '\n' and '\t' to ','
#also changes the other repeated headers to nothing
af = pd.DataFrame(loca,columns =['area'])
af.area = af.area.apply(lambda x: x.replace('\n', ','))
af.area = af.area.apply(lambda x: x.replace('\t', ','))
af.area = af.area.apply(lambda x: x.replace('LGA:,', ''))
af.area = af.area.apply(lambda x: x.replace(',POLLING UNIT:', ''))
af.area = af.area.apply(lambda x: x.replace(',WARD:', ''))
af.area = af.area.apply(lambda x: x.replace(',DELIM:', ''))
af.area = af.area.apply(lambda x: x.replace(',RECORDS:', ''))

af = af['area'].str.split(",", n = 5, expand = True)
af.columns = ['LGA', 'Polling_unit', 'Ward', 'Delim','Records']
af

,LGA,Polling_unit,Ward,Delim,Records
0,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968


In [341]:
import pandas as pd

df = pd.DataFrame(names,columns =['Names'])
df = df.replace('\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0', '')
df.Names = df.Names.apply(lambda x: x.replace('\t\n\t', ','))
df.Names = df.Names.apply(lambda x: x.replace('\n\n', ','))
df.Names = df.Names.apply(lambda x: x.replace('VIN:', ''))
df.Names = df.Names.apply(lambda x: x.replace('DOB-Y:', ''))
df.Names = df.Names.apply(lambda x: x.replace('GENDER:', ','))
df = df['Names'].str.split(",", n = 5, expand = True)
df.columns = ['no_', 'First_name', 'Last_name', 'VIN','DOB','Gender']
df

,no_,First_name,Last_name,VIN,DOB,Gender
0,1,ABA,ANTHONY,90F5B1FFD8295703336,1967,M
1,2,ABANGWU,BLESSING,90F5B1FFD8295354202,1958,F
2,3,Abdu,Ado,FA5B3EC1DB523372976,1968,M
3,4,Abdulkadir,Mohadmmed D.,90F5B124EF523523270,1983,M
4,5,ABDULLAHI,ABDULSAMAD A.,INC22000000014918081,1998,M
...,...,...,...,...,...,...
963,964,Yau,Babfa,90F5AF2998522840397,1998,M
964,965,Yusuf,Blessing,90F5B0782B524224760,2000,F
965,966,Yusuf,Karibatu A.,FA5B3EC13F524082770,1993,F
966,967,ZAKARI,ISMAILA U.,90F5B1FFD8296048614,1980,F


In [338]:
import pandas as pd
#concats both df and af datafrmaes together
ed = pd.concat([df, af], axis=1)
#since the af dataframe only has one row that corresponds to the df counterpart, fill the nan values with anything above it
ed= ed.fillna(method='ffill')
ed

,no_,First_name,Last_name,VIN,DOB,Gender,LGA,Polling_unit,Ward,Delim,Records
0,1,ABA,ANTHONY,90F5B1FFD8295703336,1967,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968
1,2,ABANGWU,BLESSING,90F5B1FFD8295354202,1958,F,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968
2,3,Abdu,Ado,FA5B3EC1DB523372976,1968,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968
3,4,Abdulkadir,Mohadmmed D.,90F5B124EF523523270,1983,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968
4,5,ABDULLAHI,ABDULSAMAD A.,INC22000000014918081,1998,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968
...,...,...,...,...,...,...,...,...,...,...,...
963,964,Yau,Babfa,90F5AF2998522840397,1998,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968
964,965,Yusuf,Blessing,90F5B0782B524224760,2000,F,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968
965,966,Yusuf,Karibatu A.,FA5B3EC13F524082770,1993,F,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968
966,967,ZAKARI,ISMAILA U.,90F5B1FFD8296048614,1980,F,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968


In [339]:
from IPython.core.display import display,HTML
import pandas as pd

unwanted = {'https://cvr.inecnigeria.org/assets/img/myvoter.png','https://cvr.inecnigeria.org/img/logo.png'}
 
pictures = [ele for ele in pictures if ele not in unwanted]

ed['pic'] = pictures
# Converting links to html tags
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'
# Rendering the dataframe as HTML table
ed.to_html(escape=False, formatters=dict(pic=path_to_image_html))
# Rendering the images in the dataframe using the HTML method.
v = HTML(ed.to_html(escape=False,formatters=dict(pic=path_to_image_html)))
# Saving the dataframe as a webpage
#df.to_html('webpage.html',escape=False, formatters=dict(pic=path_to_image_html))
#a = pd.read_html('webpage.html')
#of.drop(of.loc[:, 'pictures':'Picture'].columns,inplace=True, axis=1)
v

,no_,First_name,Last_name,VIN,DOB,Gender,LGA,Polling_unit,Ward,Delim,Records,pic
0,1,ABA,ANTHONY,90F5B1FFD8295703336,1967,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968,
1,2,ABANGWU,BLESSING,90F5B1FFD8295354202,1958,F,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968,
2,3,Abdu,Ado,FA5B3EC1DB523372976,1968,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968,
3,4,Abdulkadir,Mohadmmed D.,90F5B124EF523523270,1983,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968,
4,5,ABDULLAHI,ABDULSAMAD A.,INC22000000014918081,1998,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968,
5,6,Abdullahi,Ibrahim I.,90F5B0868B522746607,1996,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968,
6,7,ABDULLAHI,JAMILU,90F5B05E53501493322,1994,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968,
7,8,Abdullahi,Mugammad B.,FA5B3EC240523551231,1986,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968,
8,9,ABDULLATEEF,QASIM O.,90F5B1FCBF504596498,1986,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968,
9,10,ABDULRAMAN,FATIMOH D.,90F5B1FCBF506416337,1988,F,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968,


# LOADING